In [17]:
import pandas as pd
import numpy as np

orders = pd.read_csv('baseNY/food_order.csv')

orders.head()

,order_id,customer_id,restaurant_name,cuisine_type,cost_of_the_order,day_of_the_week,rating,food_preparation_time,delivery_time
0,1477147,337525,Hangawi,Korean,30.75,Weekend,Not given,25,20
1,1477685,358141,Blue Ribbon Sushi Izakaya,Japanese,12.08,Weekend,Not given,25,23
2,1477070,66393,Cafe Habana,Mexican,12.23,Weekday,5,23,28
3,1477334,106968,Blue Ribbon Fried Chicken,American,29.20,Weekend,3,25,15
4,1478249,76942,Dirty Bird to Go,American,11.59,Weekday,4,25,24


In [11]:
#Pré-processamento: tratamento de dados inconsistentes ou redundantes
#Remove dados duplicados, que para fim estátisticos são irrelevantes
orders_duplicates = orders [ orders.duplicated(subset=['restaurant_name', 'cuisine_type', 'cost_of_the_order', 'day_of_the_week', 'rating', 'food_preparation_time', 'delivery_time'], keep=False)]

if len(orders_duplicates) > 0:
  print('\nAmostras redudantes ou inconsistentes:\n')
  print(orders_duplicates)
else: print('\nNão existem amostras redudantes ou inconsistentes')


Não existem amostras redudantes ou inconsistentes


In [12]:
orders_detalhes = orders.describe()


print('\nDetalhes:\n')
print(orders_detalhes)


Detalhes:

           order_id    customer_id  cost_of_the_order  food_preparation_time  \
count  1.898000e+03    1898.000000        1898.000000            1898.000000   
mean   1.477496e+06  171168.478398          16.498851              27.371970   
std    5.480497e+02  113698.139743           7.483812               4.632481   
min    1.476547e+06    1311.000000           4.470000              20.000000   
25%    1.477021e+06   77787.750000          12.080000              23.000000   
50%    1.477496e+06  128600.000000          14.140000              27.000000   
75%    1.477970e+06  270525.000000          22.297500              31.000000   
max    1.478444e+06  405334.000000          35.410000              35.000000   

       delivery_time  
count    1898.000000  
mean       24.161749  
std         4.972637  
min        15.000000  
25%        20.000000  
50%        25.000000  
75%        28.000000  
max        33.000000  
